In [1]:
import pandas as pd
import numpy as np
import re

### Check if data has the same distribution as their final dataset

In [2]:
data = pd.read_csv('dataset.csv')

In [3]:
# has the right amount of jokes & non-jokes
data.humor.value_counts()

False    100000
True     100000
Name: humor, dtype: int64

In [13]:
# split into train and test data
x_train, x_test = data['text'][:160000], data['text'][160000:]
y_train, y_test = data['humor'][:160000], data['humor'][160000:]

# cast back into dataframes 
x_train = x_train.to_frame('text')
x_test = x_test.to_frame('text')
y_train = y_train.to_frame('humor')
y_test = y_test.to_frame('humor')

In [5]:
# haven't done processing for special words (eg isn't --> is not)
data[data['text'].str.contains("isn't")].set_index('text')

,humor
text,
Me: my cat isn't overweight; she's just big-boned vet: this is a dog,True
"Did you hear, john wayne bobbit got his penis cut off again? isn't that redickless?",True
"Jesus walks into a bar no he didn't, because he isn't real.",True
Video breaks down why machismo isn't synonymous with latino men,False
George clooney's ex-anchorman dad isn't having it with sinclair,False
...,...
Miss russia 'haters' say elmira abdrazakova isn't russian enough (photos),False
It's so cool how math isn't real now that i'm a grown up.,True
There's a shockingly high chance the seafood you're eating isn't legit,False


In [6]:
all_text = ' '.join(data['text'])
all_puncs = re.findall(r'[^\w\s]', all_text)

In [7]:
all_puncs = []
for t in data['text']:
    num_puncs = re.findall(r'[^\w\s]', t)
    all_puncs.append(num_puncs)

In [8]:
len(all_puncs)

200000

In [9]:
# if we need to expand contractions ourselves

#!pip install contractions
import contractions as ct
ct.fix("Hi I'm isn't")

ModuleNotFoundError: No module named 'contractions'

## ColBERT Model

In [ ]:
#!pip install keras
#!pip install tensorflow
#!pip install --upgrade --user transformers==3.0.2 # the authors probably used version 3.0.2
#!pip install transformers
#!pip install sentencepiece

In [3]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# import tensorflow_hub as hub
import tensorflow as tf
# import bert_tokenization as tokenization
import tensorflow.keras.backend as K
from tensorflow import keras 

import os
from scipy.stats import spearmanr
from math import floor, ceil
# from transformers import *
import transformers
from transformers import TFBertModel, BertTokenizer

import seaborn as sns
import string
import re    #for regex

# import keras model layers
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Concatenate

np.set_printoptions(suppress=True)
print(tf.__version__)
print(transformers.__version__)

2.8.0
3.0.2


In [6]:
SENT_INPUT_LEN = 20
DOC_INPUT_LEN = 100
DROPOUT_RATE = 0.2

# 18 inputs, 3 for each parallel path (5 sentence-level paths & 1 document-level path)
input_sent1_1 = Input(shape=(SENT_INPUT_LEN,), name='input_ii_sent1') # input IDs
input_sent1_2 = Input(shape=(SENT_INPUT_LEN,), name='input_am_sent1') # attention masks
input_sent1_3 = Input(shape=(SENT_INPUT_LEN,), name='input_tti_sent1') # token type IDs

input_sent2_1 = Input(shape=(SENT_INPUT_LEN,), name='input_ii_sent2')
input_sent2_2 = Input(shape=(SENT_INPUT_LEN,), name='input_am_sent2')
input_sent2_3 = Input(shape=(SENT_INPUT_LEN,), name='input_tti_sent2')

input_sent3_1 = Input(shape=(SENT_INPUT_LEN,), name='input_ii_sent3')
input_sent3_2 = Input(shape=(SENT_INPUT_LEN,), name='input_am_sent3')
input_sent3_3 = Input(shape=(SENT_INPUT_LEN,), name='input_tti_sent3')

input_sent4_1 = Input(shape=(SENT_INPUT_LEN,), name='input_ii_sent4')
input_sent4_2 = Input(shape=(SENT_INPUT_LEN,), name='input_am_sent4')
input_sent4_3 = Input(shape=(SENT_INPUT_LEN,), name='input_tti_sent4')

input_sent5_1 = Input(shape=(SENT_INPUT_LEN,), name='input_ii_sent5')
input_sent5_2 = Input(shape=(SENT_INPUT_LEN,), name='input_am_sent5')
input_sent5_3 = Input(shape=(SENT_INPUT_LEN,), name='input_tti_sent5')

input_doc_1 = Input(shape=(DOC_INPUT_LEN,), name='input_ii_doc')
input_doc_2 = Input(shape=(DOC_INPUT_LEN,), name='input_am_doc')
input_doc_3 = Input(shape=(DOC_INPUT_LEN,), name='input_tti_doc')

# embedding layer for sentences and documents
#bert_embeddings = Embedding(num_tokens,embedding_dim,embeddings_initializer=keras.initializers.Constant(embedding_matrix),trainable=False)
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
#bert_embeddings = bert_model(input_ids=input_sent1_1, attention_mask=input_sent1_2, token_type_ids=input_sent1_3) ########### HELP ###########
bert_embeddings1 = bert_model(input_ids=input_sent1_1, attention_mask=input_sent1_2, token_type_ids=input_sent1_3)
bert_embeddings2 = bert_model(input_ids=input_sent2_1, attention_mask=input_sent2_2, token_type_ids=input_sent2_3)
bert_embeddings3 = bert_model(input_ids=input_sent3_1, attention_mask=input_sent3_2, token_type_ids=input_sent3_3)
bert_embeddings4 = bert_model(input_ids=input_sent4_1, attention_mask=input_sent4_2, token_type_ids=input_sent4_3)
bert_embeddings5 = bert_model(input_ids=input_sent5_1, attention_mask=input_sent5_2, token_type_ids=input_sent5_3)
bert_embeddings6 = bert_model(input_ids=input_sent6_1, attention_mask=input_sent6_2, token_type_ids=input_sent6_3)

# get pooled vectors of BERT sentence embeddings
#x1 = bert_embeddings[0]['pooled_vector'] # can also do GlobalAveragePooling1D()
#x2 = bert_embeddings[1]['pooled_vector']
#x3 = bert_embeddings[2]['pooled_vector']
#x4 = bert_embeddings[3]['pooled_vector']
#x5 = bert_embeddings[4]['pooled_vector']
#x6 = bert_embeddings[5]['pooled_vector']
x1 = bert_embeddings1[1] 
x2 = bert_embeddings2[1] 
x3 = bert_embeddings3[1]
x4 = bert_embeddings4[1]
x5 = bert_embeddings5[1]
x6 = bert_embeddings6[1] 

# fully connected layer w/ dropout
h1_1 = Dense(32, activation='relu', name="hidden1_sent1")(x1)
h1_2 = Dense(32, activation='relu', name="hidden1_sent2")(x2)
h1_3 = Dense(32, activation='relu', name="hidden1_sent3")(x3)
h1_4 = Dense(32, activation='relu', name="hidden1_sent4")(x4)
h1_5 = Dense(32, activation='relu', name="hidden1_sent5")(x5)
h1_6 = Dense(256, activation='relu', name="hidden1_doc")(x6)

h1_dropout1 = Dropout(DROPOUT_RATE, name="h1_dropout_sent1")(h1_1) ####################################################
h1_dropout2 = Dropout(DROPOUT_RATE, name="h1_dropout_sent2")(h1_2) ####################################################
h1_dropout3 = Dropout(DROPOUT_RATE, name="h1_dropout_sent3")(h1_3) #                rate TO BE CHANGED                # 
h1_dropout4 = Dropout(DROPOUT_RATE, name="h1_dropout_sent4")(h1_4) #                                                  #
h1_dropout5 = Dropout(DROPOUT_RATE, name="h1_dropout_sent5")(h1_5) ####################################################
h1_dropout6 = Dropout(DROPOUT_RATE, name="h1_dropout_doc")(h1_6)   ####################################################

# fully connected layer
h2_1 = Dense(8, activation='relu', name="hidden2_sent1")(h1_dropout1)
h2_2 = Dense(8, activation='relu', name="hidden2_sent2")(h1_dropout2)
h2_3 = Dense(8, activation='relu', name="hidden2_sent3")(h1_dropout3)
h2_4 = Dense(8, activation='relu', name="hidden2_sent4")(h1_dropout4)
h2_5 = Dense(8, activation='relu', name="hidden2_sent5")(h1_dropout5)
h2_6 = Dense(64, activation='relu', name="hidden2_doc")(h1_dropout6)

# concatenate outputs of all 6 parallel layers
xx = Concatenate()([h2_1, h2_2, h2_3, h2_4, h2_5, h2_6])

# fully connected layer w/ dropout for concatenated inputs
h3 = Dense(512, activation='relu', name="hidden3")(xx)
h3_dropout = Dropout(DROPOUT_RATE)(h3) ################ rate TO BE CHANGED ################

# fully connected layer
h4 = Dense(256, activation='relu', name="hidden4")(h3_dropout)

# final output layer
yhat = Dense(1, activation='sigmoid', name="output")(h4) # need to figure out dropout rate

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


ValueError: The first argument to `Layer.call` must always be passed.

In [2]:
#df = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/dataset.csv')
df = pd.read_csv('dataset.csv')

#df_train = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/train.csv')
df_train = pd.read_csv('train.csv')
display(df_train.head(3))
df_train = df_train[:1000]

#df_test = pd.read_csv('/kaggle/input/200k-short-texts-for-humor-detection/dev.csv')
df_test = pd.read_csv('dev.csv')
display(df_test.head(3))
df_test = df_test[:1000]

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True


,text,humor
0,What kind of cat should you take into the des...,True
1,Remember when people used to have to be in sha...,True
2,Pizza is always good. - everyone we'll see abo...,True


In [ ]:
from bertembeddings import compute_input_arrays

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
inputs      = compute_input_arrays(df_train, ['text'], tokenizer)
test_inputs = compute_input_arrays(df_test, ['text'], tokenizer)

In [ ]:
from bertembeddings import compute_input_arrays

#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#inputs      = compute_input_arrays(x_train, ['text'], tokenizer)
#test_inputs = compute_input_arrays(x_test, ['text'], tokenizer)

4178it [30:50:28,  7.29s/it]   

### each sentence-level path is differentiated sentence position in the document (maximum 5 sentences in a document)
input_sent1_1 = inputs[0] # input ids

input_sent1_2 = inputs[1] # attention masks

input_sent1_3 = inputs[2] # token type ids (of the first sentences)

input_sent2_1 = inputs[3] # input ids

input_sent2_2 = inputs[4] # attention masks

input_sent2_3 = inputs[5] # token type ids (of the 2nd sentences)

input_doc_1 = inputs[15]

input_doc_2 = inputs[16]

input_doc_3 = inputs[17] # token type ids of documents

notes
- not sure if we can just use the default axis for Concatenate (default is axis=-1)
- dropout rate is not specified (also should different dropout layers have different dropout rates?)
- output layer activation function is sigmoid right???

In [ ]:
model_inputs = [input_sent1_1, input_sent1_2, input_sent1_3,
                input_sent2_1, input_sent2_2, input_sent2_3,
                input_sent3_1, input_sent3_2, input_sent3_3,
                input_sent4_1, input_sent4_2, input_sent4_3,
                input_sent5_1, input_sent5_2, input_sent5_3,
                input_doc_1, input_doc_2, input_doc_3]
model = Model(inputs=model_inputs, outputs=yhat, name="keras_func_model") ########### HELP ###########
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics = ['accuracy']) # TO BE CHANGED

In [ ]:
input_dict = {'input_ii_sent1': inputs[0], 'input_am_sent1': inputs[1], 'input_tti_sent1': inputs[2],
              'input_ii_sent2': inputs[3], 'input_am_sent2': inputs[4], 'input_tti_sent2': inputs[5],
              'input_ii_sent3': inputs[6], 'input_am_sent3': inputs[7], 'input_tti_sent3': inputs[8],
              'input_ii_sent4': inputs[9], 'input_am_sent4': inputs[10], 'input_tti_sent4': inputs[11],
              'input_ii_sent5': inputs[12], 'input_am_sent5': inputs[13], 'input_tti_sent5': inputs[15],
              'input_ii_doc': inputs[16], 'input_am_doc': inputs[17], 'input_tti_doc': inputs[18],
}
history = model.fit(input_dict, df_train['humor'], epochs=5, batch_size=6)

In [ ]:
test_preds = model.predict(test_inputs)